In [12]:
from eventsourcing.encryption import CryptoRepository,InMemCryptoStore
from eventsourcing.repositories import EventStoreRepository
from eventsourcing.event_stores import InMemEventStore
from example.user import User
from datetime import date
from example.guid import guid

### Initialize the in-memory encryption key store, eventstore, and repositories

In [13]:
crypto_store = InMemCryptoStore()
event_store = InMemEventStore()

CryptoRepository.crypto_store = crypto_store
user_repo = EventStoreRepository[User](event_store, User)

### Create a user and show that its attributes are clear

In [14]:
user = User(guid(), "Paul", "Boulanger", date(1997,2,18))

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())

user.change_last_name("Boucher")

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())


7dc55309-1963-46dc-8b12-6fb8aa4e0c75 Paul Boulanger 1997-02-18
7dc55309-1963-46dc-8b12-6fb8aa4e0c75 Paul Boucher 1997-02-18


### Save the user in the eventstore

The eventstore and key store are both empty

In [15]:
print(event_store.current)
print(crypto_store.store)

{}
{}


Saving the user in the eventstore

In [16]:
await user_repo.save(user,0)


The events related to the user are saved in the eventstore and an encryption key linked to the user was created and stored in the encryption key store

In [17]:
print(event_store.current)
print(crypto_store.store)

{'user-7dc55309-1963-46dc-8b12-6fb8aa4e0c75': [(event:UserCreated - version:0), (event:LastNameChanged - version:1)]}
{'7dc55309-1963-46dc-8b12-6fb8aa4e0c75': b'LdPQ1wa7HZoQvGXy75HlfLShCMAQyHhyhyAUwrjqZx4='}


### Showing the encryption

The personnal data (in this case: `first_name`, `last_name`, `month_of_birth` and `day_of_birth`) are encrypted in the eventstore

In [23]:
print(event_store.current[User.to_stream_id(user.id)][0].event_data)
print(event_store.current[User.to_stream_id(user.id)][1].event_data)

{"id": "7dc55309-1963-46dc-8b12-6fb8aa4e0c75", "first_name": "encrypted_gAAAAABmpggW3WwniUc9G0y4N6g-C_nu38OyoREdlYfrRoUI67LOuLd2Ks---HbOsUOoFpz3GtrGHgjk5IZoH1Sbm-CqZB16Gw==", "last_name": "encrypted_gAAAAABmpggWxi78787QxiqGrfNWKQUE6Jqr1wxpINZlcegwF7ro1kW4Jmme9hqwKi6lxGDDkzmclSsi8v3vwfiaeD3PME9eYg==", "year_of_birth": 1997, "month_of_birth": "encrypted_gAAAAABmpggWGAZ5gICX8gPDpKmJ6A1DWGGtF_18vtnCnDITEO2WSNKpiVRwj8VfDkP2fZeqL3fx856MnDghdrZaEJMKU9sSEg==", "day_of_birth": "encrypted_gAAAAABmpggWq-PEZzgVh1glu73Y96qiLcV18sDh5ItbFwy5-Qoa_ByW3NzxhX2TPx3zTz-lhsqZkieHrJMLIRbEGEz3q42P0Q=="}
{"id": "7dc55309-1963-46dc-8b12-6fb8aa4e0c75", "last_name": "encrypted_gAAAAABmpggWAUK_UR7QI5MOui5oIsx8uZuF1Qmk6yaQA089nabzeVZ0joaQRDxSazFvKj_d12GwamCMJw-dx78xnJ0XGG7P7Q=="}


Since there is an encryption key, we can retrieve the personnal informations of the user in a decrypted state

In [19]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='7dc55309-1963-46dc-8b12-6fb8aa4e0c75', first_name='Paul', last_name='Boulanger', year_of_birth=1997, month_of_birth=2, day_of_birth=18),
 LastNameChanged(id='7dc55309-1963-46dc-8b12-6fb8aa4e0c75', last_name='Boucher')]

We delete the user's encryption key

In [20]:
crypto_store.remove(user.id)

Now the personnal informations are still encrypted

In [21]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='7dc55309-1963-46dc-8b12-6fb8aa4e0c75', first_name='encrypted_gAAAAABmpggW3WwniUc9G0y4N6g-C_nu38OyoREdlYfrRoUI67LOuLd2Ks---HbOsUOoFpz3GtrGHgjk5IZoH1Sbm-CqZB16Gw==', last_name='encrypted_gAAAAABmpggWxi78787QxiqGrfNWKQUE6Jqr1wxpINZlcegwF7ro1kW4Jmme9hqwKi6lxGDDkzmclSsi8v3vwfiaeD3PME9eYg==', year_of_birth=1997, month_of_birth='encrypted_gAAAAABmpggWGAZ5gICX8gPDpKmJ6A1DWGGtF_18vtnCnDITEO2WSNKpiVRwj8VfDkP2fZeqL3fx856MnDghdrZaEJMKU9sSEg==', day_of_birth='encrypted_gAAAAABmpggWq-PEZzgVh1glu73Y96qiLcV18sDh5ItbFwy5-Qoa_ByW3NzxhX2TPx3zTz-lhsqZkieHrJMLIRbEGEz3q42P0Q=='),
 LastNameChanged(id='7dc55309-1963-46dc-8b12-6fb8aa4e0c75', last_name='encrypted_gAAAAABmpggWAUK_UR7QI5MOui5oIsx8uZuF1Qmk6yaQA089nabzeVZ0joaQRDxSazFvKj_d12GwamCMJw-dx78xnJ0XGG7P7Q==')]

When we load our user from the eventstore, the personnal information are encrypted or replaced by fake information

In [22]:
new_user = User()
new_user.loads_from_history(events)
print(new_user.id, new_user.first_name, new_user.last_name, new_user.date_of_birth.isoformat())

7dc55309-1963-46dc-8b12-6fb8aa4e0c75 encrypted_gAAAAABmpggW3WwniUc9G0y4N6g-C_nu38OyoREdlYfrRoUI67LOuLd2Ks---HbOsUOoFpz3GtrGHgjk5IZoH1Sbm-CqZB16Gw== encrypted_gAAAAABmpggWAUK_UR7QI5MOui5oIsx8uZuF1Qmk6yaQA089nabzeVZ0joaQRDxSazFvKj_d12GwamCMJw-dx78xnJ0XGG7P7Q== 1997-01-01
